In [1]:
import httpx
import pandas as pd
import asyncio
import json
import time

In [2]:
# сразу зададим адрес сервера
server = 'http://127.0.0.1:8000'

In [3]:
# проверка первичного GET-запроса
def get_request():
    try:
        response = httpx.get(server)
        if response.status_code != 200:
            return f'Ошибка, сообщение от сервера: {response.json()[0]['detail']}'
        else:
            return f'Запрос выполнен, сообщение от сервера: {response.json()[0]['status']}'
    except ConnectionError:
        return 'Не удалось подключиться к серверу'

In [4]:
get_request()

'Запрос выполнен, сообщение от сервера: App Healty'

Подготовительный этап - реализация функций для загрузки JSON-объектов обучаемых моделей. Реализовал 2 функции:
- ручной ввод JSON-объектов через input;
- чтение JSON из файла.

Эти функции для тестирования запросов от клиента я не использую, но пусть будут. 

In [5]:
models = None

Ячейки ниже можно не запускать.

In [6]:
# функции для генерации JSON для обучения моделей
# все данные нужно вводить через input руками
def convert_to_number(x):
    try:
        x_number = float(x)
    except (TypeError, ValueError):
        return x
    else:
        return x_number


def config_json():
    try:
        models = []
        n_models = int(input('Количество обучаемых моделей:').strip())
        for i in range(n_models):
            new_model = {}
            data_link = input(f'Путь до датасета ({i+1}-я модель)')
            if data_link == '':
                raise Exception
            target = input(f'Целевая переменная ({i+1}-я модель)')
            df = pd.read_csv(data_link)
            X = df.drop(target, axis=1).values.tolist()
            y = df[target].tolist()
            mid = input(f'ID {i+1}-й модели')
            if mid == '':
                raise Exception
            mtype = input(f'Тип {i+1}-й модели')
            if mtype == '':
                raise Exception
            new_model['id'] = mid
            new_model['mtype'] = mtype
            params = {}
            param = input(f"Параметр {i+1}-й модели и его значение через пробел ('exit' для выхода)")
            while param != 'exit':
                key, value = param.split()
                params[key] = convert_to_number(value)
                param = input(f"Параметр {i+1}-й модели и его значение через пробел ('exit' для выхода)")
            new_model['hyperparameters'] = params
            models.append({'X': X, 'y': y, 'config': new_model})
    except:
        print('Ошибка ввода')
        return None
    else:
        return models

In [7]:
models = config_json()

Ошибка ввода


In [8]:
# чтение моделей из файла
def read_json():
    try:
        path_to_json = input('Введите путь до JSON-файла')
        with open(path_to_json) as json_data:
            models = json.loads(json_data.read())
            json_data.close()
    except:
        print('Ошибка ввода')
        return None
    else:
        return models

In [9]:
models = read_json() if models is None else models

Ошибка ввода


Ячейки ниже уже надо запускать.

In [10]:
# генерация примера JSON-объекта (будем использовать эту функцию дальше)
def generate_json(n_models):
    data = pd.read_csv('csv.csv')
    X = data.drop('y', axis=1).values.tolist()
    y = data['y'].tolist()
    models = []
    for i in range(n_models):
        mid = f'{i}_{time.time()}'
        mtype = 'linear' if not i % 2 else 'logreg'
        params = {}
        model_config = {'id': mid, 'ml_model_type': mtype, 'hyperparameters': params}
        models.append({'X': X, 'y': y, 'config': model_config})
    return models      

Теперь перейдем к тестированию запросов:

In [11]:
models = generate_json(4) if models is None else models # генерируем модели, если не вводили руками


# запрос на обучение моделей
def fit_request(models):
    try:
        response = httpx.post(server+'/api/v1/models/fit', json=models, timeout=None)
        msgs = response.json() 
        if response.status_code != 200:
            return f'Ошибка обучения, сообщение от сервера: {response.json()['detail']}'
        else:
            res = []
            for msg in msgs:
                res.append(f'{msg['message']}')
            return '\n'.join(res)
    except:
        return 'Ошибка обучения на стороне клиента'


# посмотрим, сколько времени занимает последовательный вызов
start = time.time()
print(fit_request(models[:2]))
print(fit_request(models[2:]))
time.time() - start

Model '0_1734438526.9829485' trained and saved
Model '1_1734438526.9829485' trained and saved
Model '2_1734438526.9829485' trained and saved
Model '3_1734438526.9829485' trained and saved


127.34460520744324

In [12]:
# асинхронный вызов fit
async def fit_request_async(models_arr):
    loop = asyncio.get_running_loop()
    tasks = [loop.run_in_executor(None, fit_request, models) for models in models_arr]
    return await asyncio.gather(*tasks)


models = generate_json(4)
start = time.time()
msg = await fit_request_async([models[:2], models[2:]])
print('\n'.join(msg))
time.time() - start

Model '0_1734438654.3473003' trained and saved
Model '1_1734438654.3473003' trained and saved
Model '2_1734438654.3473003' trained and saved
Model '3_1734438654.3473003' trained and saved


63.893426179885864

Как и ожидалось, асинхронный вызов оказался примерно в 2 раза быстрее последовательного.

In [13]:
# Загрузка в инференс
def load_request(mid):
    response = httpx.post(server+'/api/v1/models/load', json={'id': mid})
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        return f'Запрос выполнен, сообщение от сервера: {response.json()[0]['message']}'

In [14]:
mid0 = models[0]['config']['id']
mid1 = models[1]['config']['id']
mid2 = models[2]['config']['id']

# загрузка в инференс существующей модели
load_request(mid0)

"Запрос выполнен, сообщение от сервера: Model '0_1734438654.3473003' loaded"

In [15]:
# загрузка в инференс несуществующей модели
load_request('not_exist')

"Ошибка, сообщение от сервера: Cannot load 'not_exist'"

In [16]:
# загрузка в инференс сверх ограничения на количество
print(load_request(mid1))
print(load_request(mid2))

Запрос выполнен, сообщение от сервера: Model '1_1734438654.3473003' loaded
Ошибка, сообщение от сервера: Cannot load '2_1734438654.3473003'


In [17]:
# выгрузка из инференса
def unload_request(mid):
    response = httpx.post(server+'/api/v1/models/unload', json={'id': mid})
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        return f'Запрос выполнен, сообщение от сервера: {response.json()[0]['message']}'

In [18]:
# выгрузка существующей модели
unload_request(mid0)

"Запрос выполнен, сообщение от сервера: Model '0_1734438654.3473003' unloaded"

In [19]:
# выгрузка несуществующей модели
unload_request('not_exist')

"Ошибка, сообщение от сервера: Cannot unload 'not_exist'"

In [20]:
# получение состояния инференса
def get_status_request():
    response = httpx.get(server+'/api/v1/models/get_status')
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        return f'Запрос выполнен, сообщение от сервера: {response.json()[0]['status']}'

In [21]:
get_status_request()

'Запрос выполнен, сообщение от сервера: Model Status 1_1734438654.3473003 Ready'

In [22]:
# получение списка моделей с их типами
def list_models_request():
    res = []
    response = httpx.get(server+'/api/v1/models/list_models')
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        models = response.json()[0]['models']
        for model in models:
            res += [f'Модель {model['id']}, тип {model['type']}']
        return '\n'.join(res)

In [23]:
print(list_models_request())

Модель 0_1734438289.468694, тип linear
Модель 1_1734438289.468694, тип logreg
Модель 2_1734438289.468694, тип linear
Модель 3_1734438289.468694, тип logreg
Модель 0_1734438392.691605, тип linear
Модель 1_1734438392.691605, тип logreg
Модель 2_1734438392.691605, тип linear
Модель 3_1734438392.691605, тип logreg
Модель 0_1734438520.0547235, тип linear
Модель 1_1734438520.0547235, тип logreg
Модель 2_1734438520.0547235, тип linear
Модель 3_1734438520.0547235, тип logreg
Модель 0_1734438526.9829485, тип linear
Модель 1_1734438526.9829485, тип logreg
Модель 2_1734438526.9829485, тип linear
Модель 3_1734438526.9829485, тип logreg
Модель 0_1734438654.3473003, тип linear
Модель 1_1734438654.3473003, тип logreg
Модель 2_1734438654.3473003, тип linear
Модель 3_1734438654.3473003, тип logreg


Также реализуем функцию для чтения данных для предсказаний из csv-файла:

In [24]:
def get_data_for_preds():
    try:
        data_link = input('Введите путь до файлас  данными')
        df = pd.read_csv(data_link)
        target = input('Введите целевую переменную, чтобы удалить ее из данных') 
        if target in df.columns:
            df = df.drop(target, axis=1)
    except:
        return pd.read_csv('csv.csv').drop('y', axis=1).values.tolist()
    else:
        return df.values.tolist()

In [25]:
X = get_data_for_preds()

Не совсем понятно с предсказаниями: в условии задания сказано, что для инференса может быть подгружено несеолько моделей. Тогда, по идее, мы имеем право возвращать список предсказний от каждой модели. Но схема данных не предусматривает различение предсказаний по айди моделей в инференсе. 

In [26]:
# асинхронное предсказание
def predict_request(mid, X):
    response = httpx.post(server+'/api/v1/models/predict', json=[{'id': mid, 'X': X}])
    preds = []
    for r in response.json():
        preds.append(r['predictions'])
        return f'Предсказания: {preds}'


async def predict_request_async(ids, Xs):
    loop = asyncio.get_running_loop()
    tasks = [loop.run_in_executor(None, predict_request, ids[i], Xs[i]) for i in range(len(ids))]
    preds = await asyncio.gather(*tasks)
    return '\n'.join(preds)


preds = await predict_request_async([mid0, mid1], [X, X])
print(preds)

Предсказания: [[-1.1102230246251565e-16, 0.9999999999999999, 1.0]]
Предсказания: [[0.0, 1.0, 1.0]]


In [27]:
# удаление модели по айди
def remove_request(mid):
    response = httpx.delete(server+f'/api/v1/models/{mid}')
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        return f'Запрос выполнен, сообщение от сервера: {response.json()['message']}'

In [28]:
remove_request(mid0)

'Ошибка, сообщение от сервера: Not Found'

In [29]:
load_request(mid0)

"Запрос выполнен, сообщение от сервера: Model '0_1734438654.3473003' loaded"

In [30]:
# удаление всех обученных моделей
def remove_all_request():
    response = httpx.delete(server+f'/api/v1/models/remove_all')
    if response.status_code != 200:
        return f'Ошибка, сообщение от сервера: {response.json()['detail']}'
    else:
        res = [msg['message'] for msg in response.json()]
        return '\n'.join(res)

In [31]:
print(remove_all_request())

Model '0_1734438289.468694' removed
Model '1_1734438289.468694' removed
Model '2_1734438289.468694' removed
Model '3_1734438289.468694' removed
Model '0_1734438392.691605' removed
Model '1_1734438392.691605' removed
Model '2_1734438392.691605' removed
Model '3_1734438392.691605' removed
Model '0_1734438520.0547235' removed
Model '1_1734438520.0547235' removed
Model '2_1734438520.0547235' removed
Model '3_1734438520.0547235' removed
Model '0_1734438526.9829485' removed
Model '1_1734438526.9829485' removed
Model '2_1734438526.9829485' removed
Model '3_1734438526.9829485' removed
Model '0_1734438654.3473003' removed
Model '1_1734438654.3473003' removed
Model '2_1734438654.3473003' removed
Model '3_1734438654.3473003' removed


In [32]:
print(list_models_request())